In [1]:
import glob
import h5py
# import matplotlib.pyplot as plt
from multiscale_new import *
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import sys

In [2]:
train_files = glob.glob('PMMA*train.csv')
print(train_files)
test_files = glob.glob('PMMA*test.csv')
print(test_files)

['PMMAboundary_GASP_location_train.csv', 'PMMAboundary_GASP_sample_train.csv']
['PMMAboundary_GASP_location_test.csv', 'PMMAboundary_GASP_sample_test.csv']


In [3]:
qoi = 'rdot' # 'rdot' or 'qtot'
j = 3 # 0, 1, 2, or 3
k = 1 # 1, or 2
# ss = 12

In [4]:
Data = pd.read_csv(train_files[j%2])
# Data = Data.dropna()
Data = Data.to_numpy()

if qoi == 'rdot':
    Data = np.hstack((Data[:, 3:-1], Data[:, 1].reshape(-1, 1))).astype(float)
else:
    Data = np.hstack((Data[:, 3:-1], Data[:, 2].reshape(-1, 1))).astype(float)
    
max_cols = np.diag([1/max(Data[:, i]) for i in range(len(Data[0]))])
Data = np.matmul(Data, max_cols)
print(Data)
print(len(Data))

Cs = glob.glob('Cs*.npy')
print(Cs)
sparse = glob.glob('sparse*.npy')
print(sparse)
T = glob.glob('T*.npy')
print(T)

Cs = np.load(Cs[j])
sparse = np.load(sparse[j])
T = np.load(T[j])

# predict(Data[:, :-1],sparse,T,Cs)
truth = pd.read_csv(train_files[j%2]).to_numpy()
# truth = truth[:, 2]/np.max(truth[:, 2])
mean_squared_error(truth[:, k]/np.max(truth[:, k]), predict(Data[:, :-1],sparse,T,Cs))

[[0.70590027 1.         0.98737823 ... 0.76488178 0.68129824 0.06561907]
 [0.75709935 1.         0.98737823 ... 0.76488178 0.68129824 0.06550133]
 [0.76771265 1.         0.98737823 ... 0.76488178 0.68129824 0.06420685]
 ...
 [0.22179105 0.10612479 0.98494404 ... 0.68782473 0.44799568 0.        ]
 [0.22936649 0.17687465 0.98494404 ... 0.68782473 0.44799568 0.01645269]
 [0.23694193 0.24762451 0.98494404 ... 0.68782473 0.44799568 0.04733215]]
4784
['Cs__0.005_qtot_location.npy', 'Cs__0.005_qtot_sample.npy', 'Cs__0.005_rdot_location.npy', 'Cs__0.005_rdot_sample.npy']
['sparse__0.005_qtot_location.npy', 'sparse__0.005_qtot_sample.npy', 'sparse__0.005_rdot_location.npy', 'sparse__0.005_rdot_sample.npy']
['T__0.005_qtot_location.npy', 'T__0.005_qtot_sample.npy', 'T__0.005_rdot_location.npy', 'T__0.005_rdot_sample.npy']


9.276842070403885e-06

In [5]:
test = pd.read_csv(test_files[j%2]).to_numpy()
# test = test[:, 2]/np.max(truth[:, 2])
mean_squared_error(test[:, k]/np.max(truth[:, k]), predict(np.matmul(test[:, 3:-1], max_cols[:-1, :-1]),sparse,T,Cs))

0.0008260965210005038

In [6]:
Data = pd.read_csv(train_files[j%2]).to_numpy()
Data = np.array([Data[92*i+80] for i in range(52)])
mean_squared_error(Data[:, k]/np.max(truth[:, k]), predict(np.matmul(Data[:, 3:-1], max_cols[:-1, :-1]),sparse,T,Cs))

4.446932236832698e-06

In [7]:
Data = pd.read_csv(test_files[j%2]).to_numpy()
Data = np.array([Data[92*i+80] for i in range(12)])
mean_squared_error(Data[:, k]/np.max(truth[:, k]), predict(np.matmul(Data[:, 3:-1], max_cols[:-1, :-1]),sparse,T,Cs))

4.592904244531227e-05

In [8]:
52*92

4784

In [9]:
(max_cols[-1, -1]**2)

77602.84151054415

In [10]:
max_cols[-1, -1]

278.572865711189

In [11]:
1/278.572865711189

0.0035897250704838997

In [12]:
77602.84151054415*4.446932236832698e-06

0.34509457758305745